In [124]:
import tensorflow.keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, GlobalMaxPool1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os 

In [134]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [126]:
filename = '../dataset_text/miditokens.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
miditokens = [tokens.strip().split(' ') for tokens in miditokens]

In [127]:
tokenizer = Tokenizer() # token -> int
tokenizer.fit_on_texts(miditokens)

In [128]:
sample = miditokens[1]
#print(sample)
#print(tokenizer.texts_to_sequences([sample])[0]) # Example of token input -> int output
#print("Sõna sagedus kogu andmestikus", tokenizer.word_counts['end'])
#print("Sõna -> indeks teisendus", tokenizer.word_index['note:c4:v112'])
#print("Indeks -> sõna teisendus", tokenizer.index_word[55])
print(len(tokenizer.word_index))
#print(tokenizer.index_word)
#print(tokenizer.word_index)

345


In [129]:
# Turn all tokens to ints
midiTokensAsInt = tokenizer.texts_to_sequences(miditokens)
#midiTokensAsInt = np.array(midiTokensAsInt)

# Generator variant

In [8]:
## https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366
#
#def batchGenerator(trainData, VOCAB_SIZE, SEQ_LEN=30):
#    i = 0
#    while True:
#        yield loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i)
#        i += 1
#    
#            
#def loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i):
#    X = []
#    y = []
#    
#    # https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
#    
#    song = trainData[i]
#    for i in range(0, len(song) - SEQ_LEN, 1):
#        X.append(song[i:i + SEQ_LEN])
#        y.append(song[i + SEQ_LEN])
#            
#    input_len = len(X)
#    
#    # reshape the input into a format compatible with LSTM layers
#    X = np.reshape(X, (input_len, SEQ_LEN, 1))
#    
#    # normalize input
#    X = X / float(VOCAB_SIZE)
#    y = to_categorical(y, num_classes = VOCAB_SIZE)
#    
#    return (X, y)
#
#VOCAB_SIZE = len(tokenizer.word_index)
#SEQ_LEN = 20
#
#batchGen = batchGenerator(midiTokensAsInt, VOCAB_SIZE, SEQ_LEN)

In [ ]:
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(20, 1),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
#filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
#checkpoint = ModelCheckpoint(
#    filepath, monitor='loss', 
#    verbose=0,        
#    save_best_only=True,        
#    mode='min'
#)    
#callbacks_list = [checkpoint]     
##model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
#
#model.fit(batchGen, 
#          epochs=200,
#          callbacks=callbacks_list,
#          verbose=1)

# Non-generator variant

In [139]:
X = []
y = []

VOCAB_SIZE = len(tokenizer.word_index)
SEQ_LEN = 100

# https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

for song in midiTokensAsInt:
    for i in range(0, len(song) - SEQ_LEN, 1):
        X.append(song[i:i + SEQ_LEN])
        y.append(song[i + SEQ_LEN])

input_len = len(X)

# reshape the input into a format compatible with LSTM layers
X = np.reshape(X, (input_len, SEQ_LEN, 1))

# normalize input
X = X / float(VOCAB_SIZE)
y = to_categorical(y)

In [ ]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE+1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "../weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X, y, epochs=200, batch_size=64, callbacks=callbacks_list)

In [140]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE+1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Load the weights to each node
model.load_weights('../weights/weights.hdf5')

In [150]:
start = np.random.randint(0, len(X)-1)
intToNote = dict(map(reversed, tokenizer.word_index.items()))

pattern = midiTokensAsInt[222][:100]
print(pattern)
#pattern = [i for [i] in pattern]
prediction_output = []
for tokenInt in pattern:
    prediction_output.append(intToNote[tokenInt])

# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(VOCAB_SIZE)
    
    prediction = model.predict(prediction_input, verbose=0)
    
    index = np.argmax(prediction)
    result = intToNote[index]
    prediction_output.append(result)
    
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

[154, 213, 207, 3, 16, 43, 47, 47, 35, 35, 2, 17, 4, 2, 48, 48, 5, 26, 104, 104, 2, 27, 105, 105, 14, 72, 72, 2, 15, 73, 73, 26, 104, 104, 2, 27, 4, 2, 36, 36, 105, 105, 5, 24, 72, 72, 62, 62, 2, 25, 8, 2, 73, 73, 9, 54, 118, 118, 2, 55, 6, 2, 119, 119, 7, 54, 118, 118, 2, 55, 8, 2, 63, 63, 119, 119, 9, 32, 104, 104, 56, 56, 2, 33, 10, 2, 105, 105, 11, 4, 2, 5, 26, 72, 72, 2, 27, 73, 73, 4]


In [152]:
text2midi(miditokens[222]).show("midi")

In [154]:
#print(prediction_output)
genmidistream = text2midi(prediction_output)
genmidistream.show("midi")
#fp = genmidistream.write('midi', fp='uhhh.mid')

In [144]:
from music21 import *

def text2midi(tokens):
    s = stream.Stream()
    
    currentVelocity = 0

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))
            
        if token.startswith("velocity"):
            currentVelocity = int(splitToken[1])

        if token.startswith("note") and not token.lower().endswith("off"):
            noteDuration = 0
            noteName = splitToken[1]

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.lower().endswith("off")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = currentVelocity
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s

#text2midi(midiTokens).show("midi")